In [1]:
from dataset.beetlebox import BeetleBox
from dataset.swebench import SWEBench

/Users/karamustafae/Projects/bug_localization/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# instance = BeetleBox()
instance = SWEBench()

In [7]:
bug_instances = instance.get_bug_instances(sample_size=500)

In [9]:
len(bug_instances)

498

In [6]:
from method.openrouter_localizer import OpenRouterLocalizer
localizer = OpenRouterLocalizer()

Invalid model specification: qwen-coder-32b, using default: qwen/qwen3-coder:free


In [8]:
localizer.localize(bug_instance)

OpenAILocalizerResponse(candidate_files=['astropy/wcs/wcsapi/fitswcs.py', 'astropy/wcs/wcs.py', 'astropy/wcs/wcsapi/high_level_wcs_wrapper.py', 'astropy/wcs/wcsapi/wrappers/sliced_wcs.py', 'astropy/visualization/wcsaxes/core.py', 'astropy/wcs/wcsapi/high_level_api.py', 'astropy/wcs/wcsapi/low_level_api.py', 'astropy/visualization/wcsaxes/wcsapi.py', 'astropy/wcs/wcsapi/utils.py', 'astropy/wcs/wcsapi/wrappers/base.py'])

In [9]:
bug_instance.ground_truths

['astropy/wcs/wcsapi/fitswcs.py']

## Full Evaluation

In [16]:
instance = SWEBench()

In [18]:
bug_instances = instance.get_bug_instances()

In [22]:
import json
import os
from method.openai_localizer import OpenAILocalizer
from method.openrouter_localizer import OpenRouterLocalizer

output_dir = "responses"
os.makedirs(output_dir, exist_ok=True)

localizer = OpenAILocalizer()
responses = {}
import random
index = random.randint(0, len(bug_instances)-1)
for i, bug in enumerate(bug_instances[index:index+50]): 
    response = localizer.localize(bug)
    data = {
        'bug_id': bug.instance_id,
        'ground_truths': bug.ground_truths,
        'response': response
    }
    
    filename = f"{output_dir}/response_{bug.instance_id}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False, default=str)
    
    responses[bug.instance_id] = data

Error processing chunk 3: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Error during aggregation: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


Exception: Request failed: 429, {
    "error": {
        "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.",
        "type": "insufficient_quota",
        "param": null,
        "code": "insufficient_quota"
    }
}


In [ ]:
with open("localization_results.json", "w", encoding="utf-8") as f:
    json.dump(responses, f, indent=4, ensure_ascii=False)

In [ ]:
evaluator = Evaluator()
results = evaluator.evaaluate(responses)